In [2]:
# Cell 1: Environment Check
import torch
import sys
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
PyTorch version: 2.6.0+cu118
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.00 GB


In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA")

2.6.0+cu118
True
NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from model_attention import Encoder, Decoder, Seq2Seq
from data_loader import load_data, create_tokenizers, TranslationDataset
import numpy as np
import time
import math
from tqdm import tqdm
from torch.utils.data import DataLoader


Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.00 GB


In [5]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
def train(model, iterator, optimizer, criterion, clip, device):
    model.train()
    epoch_loss = 0
    
    for batch in tqdm(iterator, desc='Training'):
        src = batch['source'].to(device)
        trg = batch['target'].to(device)
        
        optimizer.zero_grad()
        output = model(src, trg)
        
        # Remove <sos> token
        output = output[:, 1:].reshape(-1, output.shape[-1])
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [7]:
def evaluate(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(iterator, desc='Evaluating'):
            src = batch['source'].to(device)
            trg = batch['target'].to(device)
            
            output = model(src, trg, 0)  # Turn off teacher forcing
            
            # Remove <sos> token
            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

In [8]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [9]:
def train_model(model, train_loader, val_loader, n_epochs, learning_rate, clip, device):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # 0 is padding index
    
    best_valid_loss = float('inf')
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        train_loss = train(model, train_loader, optimizer, criterion, clip, device)
        valid_loss = evaluate(model, val_loader, criterion, device)
        
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'attention_model.pt')
        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


In [ ]:
if __name__ == "__main__":
    # Hyperparameters
    HIDDEN_DIM = 256
    N_LAYERS = 2
    DROPOUT = 0.2
    BATCH_SIZE = 64
    N_EPOCHS = 10
    CLIP = 1
    LEARNING_RATE = 0.001
    VOCAB_SIZE = 8000

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Load and prepare data
    train_df, val_df = load_data('english_assamese.csv')
    source_tokenizer, target_tokenizer = create_tokenizers(train_df, vocab_size=VOCAB_SIZE)
    
    # Get vocabulary sizes from tokenizers
    INPUT_DIM = source_tokenizer.get_piece_size()
    OUTPUT_DIM = target_tokenizer.get_piece_size()
    
    # Initialize model
    enc = Encoder(INPUT_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT)
    dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT)
    model = Seq2Seq(enc, dec)
    
    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Create datasets
    train_dataset = TranslationDataset(
        train_df['eng'].tolist(),
        train_df['asm'].tolist(),
        source_tokenizer,
        target_tokenizer
    )
    
    val_dataset = TranslationDataset(
        val_df['eng'].tolist(),
        val_df['asm'].tolist(),
        source_tokenizer,
        target_tokenizer
    )
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    # Train the model
    train_model(model, train_loader, val_loader, N_EPOCHS, LEARNING_RATE, CLIP, device) 

Using device: cuda
Loading data from english_assamese.csv
Total samples: 87849
Training samples: 70279
Validation samples: 17570
Creating tokenizers...
Training English tokenizer...
Training Assamese tokenizer...
Tokenizers created successfully!
